# Text Classification with Recurrent Neural Networks, DBPedia dataset

Modified from original code here: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/learn/text_classification.py

### Make the notebook compatible with both Python 2 and 3

http://python-future.org/compatible_idioms.html

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

### Plot graphs inline

In [3]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [4]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)
print(matplotlib.__version__)

1.3.0
1.13.1
0.20.1
2.0.2


In [5]:
EMBEDDING_SIZE = 50
n_words = 0
MAX_LABEL = 15
WORDS_FEATURE = 'words'  # Name of the input words feature.

### Download the DBPedia dataset

http://wiki.dbpedia.org/Datasets

The dataset contains first paragraph of the wikipedia page for ~0.5M entities and the label is on of 15 categories. This is *topic classification*, each paragraph is labelled with the topic that it deals with.

Examples of topics:

* People
* Companies
* Creative Works
* Universities/Colleges
* Creative Works
* etc.

In [6]:
dbpedia = tf.contrib.learn.datasets.load_dataset(
      'dbpedia', size='medium', test_with_fake_data=False)

In [7]:
len(dbpedia.train.data)

560000

In [8]:
len(dbpedia.test.data)

70000

### Data from DBPedia should be shuffled before use

* The data will be ordered, paragraphs from topic 1 will be at the beginning, topic 2 will be after that and so on
* Shuffle the data before you use it for training or test

In [9]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(dbpedia.train.data)))

data_shuffled = dbpedia.train.data[shuffle_indices]
target_shuffled = dbpedia.train.target[shuffle_indices]

In [12]:
TRAIN_DATA = 500

train_data = data_shuffled[:TRAIN_DATA]
train_target = target_shuffled[:TRAIN_DATA]

In [13]:
shuffle_indices = np.random.permutation(np.arange(len(dbpedia.test.data)))

data_shuffled = dbpedia.test.data[shuffle_indices]
target_shuffled = dbpedia.test.target[shuffle_indices]

In [14]:
TEST_DATA = 50

test_data = data_shuffled[:TEST_DATA]
test_target = target_shuffled[:TEST_DATA]

In [15]:
train_data[:5]

array([['Darbandikhan',
        ' Darbandikhan an town center and the judiciary in the province of Sulaimaniya northern Iraq. It is situated within the area of autonomy for the Kurdistan region of Iraq inhabited by the majority of Kurds. Darbandikhan is located close to Darbandikhan Lake (35.152477\xc2\xb0N 45.795021\xc2\xb0E\xef\xbb\xbf / 35.152477; 45.795021) and on the border with Diyala Province.'],
       ['Dehkord Isfahan',
        ' Dehkord (Persian: \xd8\xaf\xd9\x87\xd9\x83\xd8\xb1\xd8\xaf\xe2\x80\x8e; also known as Deh Kord \xe2\x80\x98Al\xc4\xab\xc4\x81b\xc4\x81d and Kordkand\xc4\xab) is a village in Vardasht Rural District in the Central District of Semirom County Isfahan Province Iran. At the 2006 census its population was 182 in 44 families.'],
       ['Virginie-class frigate',
        ' The Virginie class was a class of ten 44-gun frigates of the French Navy designed in 1793 by Jacques-No\xc3\xabl San\xc3\xa9. An eleventh vessel (Zephyr) begun in 1794 was never completed.

### Find the number of unique topics in the training set

In [17]:
np.unique(dbpedia.train.target)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int32)

In [19]:
x_train = pd.Series(train_data[:,1])
y_train = pd.Series(train_target)
x_test = pd.Series(test_data[:,1])
y_test = pd.Series(test_target)

### Word embeddings

In order to perform sentiment analysis or classifcation on text, each word is generally represented as an encoding. The encodings are typically done in such a way that words which appear together have codes which are close to each other.

Am encoding can be a numeric embedding (a low-dimensional representation), one-hot representation etc.

Word embeddings all have the same dimensionality which you can specify. A document is a vector of word embeddings (one dbpedia instance is a document in this case)

* Each document should be of the **same length**, we choose the length of the document to be equal to the length of the longest document
* The other documents will be **padded** by a special symbol to be the same max length

In [20]:
max_document_length_train = max([len(x.split(" ")) for x in x_train])
max_document_length_test = max([len(x.split(" ")) for x in x_test])

print(max_document_length_train, max_document_length_test)

95 86


In [21]:
MAX_DOCUMENT_LENGTH = max(max_document_length_train,
                          max_document_length_test)

### Vocabulary processor

http://tflearn.org/data_utils/

Library to map every word which occurs in our dataset to a unique identifer. If there are 10023 words each will be assigned a unique id from 1-10023

In [22]:
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
      MAX_DOCUMENT_LENGTH)

In [23]:
x_train = np.array(list(vocab_processor.fit_transform(x_train)))
x_test = np.array(list(vocab_processor.transform(x_test)))

In [24]:
x_train[:3]

array([[  1,   2,   3,   4,   5,   6,   7,   8,   6,   9,  10,  11,  12,
         13,  14,  15,  16,  17,   6,  18,  10,  19,  20,   6,  21,  22,
         10,  13,  23,  24,   6,  25,  10,  26,   1,  15,  27,  28,  29,
          1,  30,  31,  32,  33,  34,  35,  36,  31,  37,  34,  38,   5,
         39,   6,  40,  41,  42,  43,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [ 44,  45,  46,  47,  47,  46,  46,  48,  49,  50,  51,  52,  53,
         48,  54,  55,  56,  57,   5,  58,  15,  59,  60,   8,  61,  62,
         63,   8,   6,  64,  63,  10,  65,  66,  67,  43,  68,  69,   6,
         70,  71,  72,  73,  74,  75,   8,  76,  77,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,

In [26]:
train_data[:1]

array([['Darbandikhan',
        ' Darbandikhan an town center and the judiciary in the province of Sulaimaniya northern Iraq. It is situated within the area of autonomy for the Kurdistan region of Iraq inhabited by the majority of Kurds. Darbandikhan is located close to Darbandikhan Lake (35.152477\xc2\xb0N 45.795021\xc2\xb0E\xef\xbb\xbf / 35.152477; 45.795021) and on the border with Diyala Province.']],
      dtype='|S4112')

In [27]:
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 7023


In [28]:
tf.reset_default_graph()

### EstimatorSpec defines the model you run

https://www.tensorflow.org/api_docs/python/tf/estimator/EstimatorSpec

Helper function to set up the estimator spec for prediction, training as well as evaluation of your model.

* *logits* output layer to which softmax activation is to be applied
* *labels* training labels used to calculate the loss (we convert these labels to one-hot notation)
* *mode* prediction, training or evaluation

In [30]:
def estimator_spec_for_softmax_classification(logits, labels, mode):
    
    # Prediction mode
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions= {
                'class': predicted_classes,
                'prob': tf.nn.softmax(logits)
            })

    # Training mode
    onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

    # Evaluation mode
    eval_metric_ops = {
      'accuracy': tf.metrics.accuracy(
          labels=labels, predictions=predicted_classes)
    }

    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### Define the RNN model

* *features* the training dataset features
* *labels* the training labels
* *mode* prediction, training or evaluation to be passed to the estimator spec method

In [31]:
def rnn_model(features, labels, mode):
    # Convert indexes of words into embeddings.

    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    word_vectors = tf.contrib.layers.embed_sequence(
      features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)

    # Flatten esults to be a list of tensors [batch_size, EMBEDDING_SIZE]. This
    # is just a list of words
    word_list = tf.unstack(word_vectors, axis=1)

    # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

    # Create an unrolled Recurrent Neural Networks to length of
    # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

    # If you use a dynamic RNN you do not need to unstack the word_vectors,
    # it will be done under the hood for you
    #   _, encoding = tf.nn.dynamic_rnn(cell, word_vectors, dtype=tf.float32)

    # Given encoding of RNN, take encoding of last step (e.g hidden size of the
    # neural network of last step) and pass it as features for softmax
    # classification over output classes.

    logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)
    return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)

### Instantiate an estimator with the RNN model

In [32]:
classifier = tf.estimator.Estimator(model_fn=rnn_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/var/folders/yd/1rlyjfk975d3bb98d7_nyt740000gn/T/tmplB77fP', '_save_summary_steps': 100}


### Train the RNN model

* The *numpy_input_fn* allows you to specify features and labels in the form of numpy arrays
* We use full batch for training
* Run for one epoch and shuffle the data

In [33]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_train},
      y=y_train,
      batch_size=len(x_train),
      num_epochs=None,
      shuffle=True)
classifier.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/yd/1rlyjfk975d3bb98d7_nyt740000gn/T/tmplB77fP/model.ckpt.
INFO:tensorflow:loss = 2.70726, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/yd/1rlyjfk975d3bb98d7_nyt740000gn/T/tmplB77fP/model.ckpt.
INFO:tensorflow:Loss for final step: 0.216361.


### Make predictions on test data

In [40]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
predictions = classifier.predict(input_fn=test_input_fn)

### Reshape predicted data to be the same as original labels

This will allow us to compare the results for accuracy

In [41]:
y_predicted = np.array(list(p['class'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_test).shape)

INFO:tensorflow:Restoring parameters from /var/folders/yd/1rlyjfk975d3bb98d7_nyt740000gn/T/tmplB77fP/model.ckpt-100


In [42]:
y_predicted

array([11,  3, 14,  5,  2,  3, 14, 14, 13, 11,  1,  5,  8,  6, 11,  3,  3,
        1,  9,  1,  3, 14,  2,  5,  5, 14,  7,  2,  8,  3,  4,  9,  5, 10,
       14, 10, 14,  6,  8, 11,  6,  9,  5, 14,  5,  3, 13, 11,  9, 11])

### Measure accuracy using scikit-learn and tensorflow

In [45]:
from sklearn import metrics

score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy (sklearn): {0:f}'.format(score))

Accuracy (sklearn): 0.280000


In [46]:
scores = classifier.evaluate(input_fn=test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))

INFO:tensorflow:Starting evaluation at 2017-09-08-07:29:07
INFO:tensorflow:Restoring parameters from /var/folders/yd/1rlyjfk975d3bb98d7_nyt740000gn/T/tmplB77fP/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-09-08-07:29:08
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.28, global_step = 100, loss = 2.47707
Accuracy (tensorflow): 0.280000
